## Создание классических моделей машинного обучения

### Импорт собранных и обработанных данных

In [33]:
import pandas as pd

# Для mac
df_nlp = pd.read_csv(r'/Users/user/Documents/ML.csv')

# Для win
#df_nlp = pd.read_csv(r'C:\Users\User\Downloads\ML.csv'), если в начале лишних столбец с индексами

#df_ml = df_nlp.drop(df_nlp.columns[[0,3,4,5,6,7,8,9,10]], axis = 1)

df_ml

,main_rubric,rubric,text_lemm,title_lemm
0,Происшествия,Происшествия,следствие попадать глава аналитический отдел п...,обвинение взяточничество арестовывать подмоско...
1,Происшествия,Происшествия,систематическое получение взятка арестовывать ...,систематическое получение взятка арестовывать ...
2,Происшествия,Происшествия,место выезжать аварийный бригада сотрудник адм...,приграничное село белгородский область попадат...
3,Происшествия,Происшествия,село старый хутор белгородский область попадат...,украинские войско обстреливать село белгородск...
4,Происшествия,Происшествия,четверг декабрь обстрел вооруженный формирован...,губернатор гладков всу обстреливать село стары...
...,...,...,...,...
3391,Наука,Космос,ученые томский университет система управление ...,томские ученый разрабатывать система электропи...
3392,Наука,Космос,вес бортовой система космический аппарат число...,томске разрабатывать система электропитание сп...
3393,Наука,Космос,двадцать пятый август год зарегистрировать мар...,ученые зафиксировать марсотрясение рекордный силы
3394,Наука,Космос,сейсмометр марсианский модуль insight зафиксир...,марсе происходить крупный землетрясение истори...


### Кодирование целевой переменной. Формирование тестовой и обучающей выборок

In [47]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

rubrics = ['Политика', 'Общество', 'Экономика', 'В мире', 'Спорт', 'Происшествия', 'Культура', 'Технологии', 'Наука']

encoder = LabelEncoder()
rubrics_list = df_ml['main_rubric'].to_list()
rubric_labels = encoder.fit_transform(rubrics_list)

X = df_ml['text_lemm']
y = rubric_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
my_tags = rubrics

X_train.shape

(2377,)

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

### Байесовский классификатор

In [36]:
from sklearn.naive_bayes import MultinomialNB

def nb_classifier():
    
    nb = Pipeline ([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
                   ])

    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred, target_names=my_tags))
    
    return round(accuracy_score(y_pred, y_test), 2)

nb_classifier()

accuracy 0.8056918547595682
              precision    recall  f1-score   support

    Политика       0.00      0.00      0.00        35
    Общество       0.97      0.86      0.91       124
   Экономика       1.00      0.41      0.58        78
      В мире       0.62      0.97      0.75       213
       Спорт       0.91      0.68      0.78       108
Происшествия       0.97      0.56      0.71        68
    Культура       0.79      0.99      0.88       153
  Технологии       0.95      0.84      0.89       106
       Наука       0.90      0.93      0.91       134

    accuracy                           0.81      1019
   macro avg       0.79      0.69      0.71      1019
weighted avg       0.82      0.81      0.79      1019



/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.81

### Метод опорных векторов

In [37]:
from sklearn.linear_model import SGDClassifier

def sgd_classifier():
    
    sgd = Pipeline ([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                    ])

    sgd.fit(X_train, y_train)
    y_pred = sgd.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred, target_names=my_tags))
    
    return round(accuracy_score(y_pred, y_test), 2)

sgd_classifier()

accuracy 0.8851815505397449
              precision    recall  f1-score   support

    Политика       0.57      0.46      0.51        35
    Общество       0.94      0.94      0.94       124
   Экономика       0.97      0.87      0.92        78
      В мире       0.88      0.90      0.89       213
       Спорт       0.88      0.85      0.86       108
Происшествия       0.92      0.72      0.81        68
    Культура       0.89      0.98      0.93       153
  Технологии       0.87      0.92      0.89       106
       Наука       0.86      0.92      0.89       134

    accuracy                           0.89      1019
   macro avg       0.87      0.84      0.85      1019
weighted avg       0.88      0.89      0.88      1019



0.89

### Логистическая регрессия

In [38]:
from sklearn.linear_model import LogisticRegression

def logreg_classifier():

    logreg = Pipeline ([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                       ])

    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred, target_names=my_tags))
    
    return round(accuracy_score(y_pred, y_test), 2)

logreg_classifier()

accuracy 0.8792934249263984
              precision    recall  f1-score   support

    Политика       0.53      0.49      0.51        35
    Общество       0.96      0.93      0.94       124
   Экономика       0.96      0.85      0.90        78
      В мире       0.86      0.90      0.88       213
       Спорт       0.84      0.85      0.85       108
Происшествия       0.84      0.79      0.82        68
    Культура       0.93      0.97      0.95       153
  Технологии       0.89      0.89      0.89       106
       Наука       0.87      0.88      0.88       134

    accuracy                           0.88      1019
   macro avg       0.85      0.84      0.84      1019
weighted avg       0.88      0.88      0.88      1019



/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.88

### Дерево решений 

In [39]:
from sklearn.tree import DecisionTreeClassifier

def dtree_classifier():
    
    dtree = Pipeline ([('vect', CountVectorizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', DecisionTreeClassifier(max_depth = 2)),
                      ])

    dtree.fit(X_train, y_train)
    y_pred = dtree.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred, target_names=my_tags))
    
    return round(accuracy_score(y_pred, y_test), 2)

dtree_classifier()

accuracy 0.25318940137389595
              precision    recall  f1-score   support

    Политика       0.00      0.00      0.00        35
    Общество       1.00      0.13      0.23       124
   Экономика       0.00      0.00      0.00        78
      В мире       0.22      1.00      0.36       213
       Спорт       0.00      0.00      0.00       108
Происшествия       1.00      0.06      0.11        68
    Культура       0.81      0.16      0.27       153
  Технологии       0.00      0.00      0.00       106
       Наука       0.00      0.00      0.00       134

    accuracy                           0.25      1019
   macro avg       0.34      0.15      0.11      1019
weighted avg       0.36      0.25      0.15      1019



/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.25

### Метод K ближайщих соседей KNN

In [40]:
from sklearn.neighbors import KNeighborsClassifier

def knn_classifier():

    knn = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', KNeighborsClassifier(n_neighbors=3)),
                   ])

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
    print(classification_report(y_test, y_pred, target_names=my_tags))
    
    return round(accuracy_score(y_pred, y_test), 2)

knn_classifier()

accuracy 0.8557409224730128
              precision    recall  f1-score   support

    Политика       0.34      0.54      0.42        35
    Общество       0.86      0.95      0.90       124
   Экономика       0.84      0.86      0.85        78
      В мире       0.87      0.82      0.84       213
       Спорт       0.84      0.83      0.84       108
Происшествия       0.98      0.82      0.90        68
    Культура       0.93      0.95      0.94       153
  Технологии       0.95      0.82      0.88       106
       Наука       0.87      0.87      0.87       134

    accuracy                           0.86      1019
   macro avg       0.83      0.83      0.83      1019
weighted avg       0.87      0.86      0.86      1019



0.86

### Метод градиентного бустинга

In [41]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,r2_score 

def gbr_classifier():
    
    gbr = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', GradientBoostingRegressor(n_estimators=200, max_depth=12, criterion='squared_error', learning_rate=0.03, min_samples_leaf=16, min_samples_split=16)),
                   ])

    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    train_accuracy_score=gbr.score(X_train,y_train)
    test_accuracy_score=gbr.score(X_test,y_test)
    mse = mean_squared_error(y_test,y_pred)

    print(train_accuracy_score)
    print(test_accuracy_score)
    print("MSE: %.2f" % mse)
    print(r2_score(y_test,y_pred))
    
    return round(test_accuracy_score, 2)

gbr_classifier()

0.5237779153699518
0.41224377930584766
MSE: 3.38
0.41224377930584766


0.41